<a href="https://colab.research.google.com/github/atc857/chi2anova/blob/main/Aula5_Testes_Hipoteses_chi2_Anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testes de hipóteses utilizando Python

Disciplina: Metodologia Científica 2 (Métodos Quantitativos)

por André T. Campos

O pacote `SciPy` possui ferramentas dedicadas à resolução de problemas típicos de computação científica. Nesta aula utilizaremos especificamente o módulo `stats`, que possui uma vasta gama de funcionalidades para lidar com conceitos estatísticos fundamentais. Para maiores informações sobre as funcionalidades disponíveis, consulte a documentação no link https://docs.scipy.org/doc/scipy/reference/stats.html.

![SciPy logo](https://www.fullstackpython.com/img/logos/scipy.png)

In [ ]:
# Atualização da biblioteca scipy para a versão mais recente disponível.
# Google Colab já vem com a biblioteca instalada, porém, pode não ser a versão mais recente. Portanto, é recomendável atualizar.
!pip install --upgrade scipy

In [ ]:
# Importação das bibliotecas necessárias
from scipy import stats
from scipy import optimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Configuração do estilo dos gráficos
sns.set(style='ticks')

In [ ]:
# reprodutibilidade
np.random.seed(10)

# Teste $\chi ^2$ para independência entre variáveis

O objetivo do teste é avaliar se existe associação entre variáveis qualitativas categóricas.

Suposições:
*   Todas as frequências esperadas são acima de 5.
*   Os dados são mutuamente exclusivos.

---


**Ex. 4)** Aplicou-se um questionário aos bombeiros militares do Distrito Federal para aferir o grau de concordância com determinada proposta. As respostas possíveis eram "concorda", "discorda" e "neutro". Foram definidas 4 categorias ("Cabos e Soldados", "Subtenentes e Sargentos", "Capitães e Tenentes" e "Oficiais Superiores"). As 1.000 respostas estão dispostas na tabela abaixo. Deseja-se verificar se há associação entre a patente do entrevistado e a resposta dada, ao nível de significância de 5%.

*   A estatística teste é dada por \\[ \chi^2 = \sum \frac{ (\mathcal{O} - \mathcal{E})^2}{ \mathcal{E} } \\]
*   Hipótese nula: \\( H_0: \pi_1 = \pi_2 \\) (variáveis independentes)
*   Hipótese alternativa: \\( H_1: \pi_1 \neq \pi_2 \\) (existe associação entre as variáveis)

In [ ]:
# Cria base com respostas e respondentes a um questionário de concordância a quesito
respondentes = np.random.choice(a= ["CB_SD","SGT_ST","TEN_CAP","OF_SUP"],
                              p = [0.15, 0.55 ,0.25, 0.05],
                              size=1000)

respostas = np.random.choice(a= ["CONCORDA","DISCORDA","NEUTRO"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

# Transforma em dataframe as respostas e respondentes
resultado = pd.DataFrame({"patente":respondentes, 
                       "concordancia":respostas})

resultado.head()

In [ ]:
# Cria uma tabela de contigência (crosstab)
result_tab = pd.crosstab(resultado.patente, resultado.concordancia, margins = True)

# Renomeia as colunas
result_tab.columns = ["CONCORDA","DISCORDA","NEUTRO","TOTAL"]

# Renomeia as linhas
result_tab.index = ["CB_SD","SGT_ST","TEN_CAP","OF_SUP","TOTAL"]

result_tab

In [ ]:
# Vamos salvar a tabela sem os totais para usar no teste
result_tab = result_tab.iloc[0:4,0:3]
result_tab

Usando o módulo `stats`, basta chamar o comando `scipy.stats.chi2_contingency(observed, correction=True, lambda_=None)`, em que:
*   `observed` são os dados sob análise numa tabela de contingência
*   `correction=True` o padrão `True` aplica a correção de Yates em tabelas 2x2.
*   `lambda_` permite aplicar estatística teste diferente da qui-quadrado estático de Pearson.

A saída do comando é a estatística do teste (\\(\chi^2 )\\), o valor-p, os graus de liberdade e a frequência esperada (mesma dimensão da tabela de contingência).

In [ ]:
chi2, p, gl, expect = stats.chi2_contingency(result_tab)
chi2,p

In [ ]:
# nível de significância
alpha = 0.05
# Imprime o resultado do teste
print(f'A estatística do teste foi qui-quadrado={chi2:.4f}, correspondente ao valor-p de {p:.3%}.')
if p <= alpha:
  print(f'Hipótese nula rejeitada ao nível de significância de {alpha:.1%}. Portanto, há associação entre as variáveis estudadas.')
else:
  print(f'Não há evidência suficiente para rejeitar a hipótese nula em favor da hipótese alternativa. Ou seja, assume-se que as variáveis são independentes.')

## Análise alternativa do resultado do teste (pelo valor da estatística do teste).
<img src="https://2012books.lardbucket.org/books/beginning-statistics/section_15/34d06306c2e726f6d5cd7479d9736e5e.jpg"  width="50%" height="40%">

In [ ]:
# Valor crítico tabelado de qui-quadrado ao nível de significância definido
chi2tab = stats.chi2.ppf(1-alpha, gl)
chi2tab

In [ ]:
# Se o valor do teste (chi2) cair acima do valor crítico tabelado (chi2tab), está na região crítica
if chi2 > chi2tab:
  print(f'Hipótese nula rejeitada, pois qui-quadrado={chi2:.4f} está na região crítica ao nível de significância de {alpha:.1%}.')
else:
  print(f'Não há evidência suficiente para rejeitar a hipótese nula, pois qui-quadrado={chi2:.4f} está na região de aceitação ao nível de significância de {alpha:.1%}.')

# Análise de variância (ANOVA)

O teste **F** é adequado para determinar se as médias de duas ou mais populações são iguais.

Suposições:
*   As amostras devem ser aleatórias e independentes;
*   As amostras devem ser extraídas de populações normais;
*   As populações devem ter variâncias iguais.



---


**Ex. 5)** No mesmo questionário do exemplo anterior também foi perguntado o peso (massa em kg) do entrevistado. Deseja-se aferir se as médias dos pesos são iguais para todos os grupos investigados.

*   A estatística teste é dada por \\[ \mathrm{F}= \frac{S_b^2}{S_w^2} = \frac{ \frac{ \sum_{j} n_j (\bar{x}_j - \bar{\bar{x}})^2}{ k-1 } } { \frac{\sum_{ij} n_j (x_{ij} - \bar{x}_j)^2}{\sum_{j}n_j - k} } \\]
*   Hipótese nula: \\( H_0: \mu_1 = \mu_2 = \cdots  = \mu_k \\) (todas as médias são iguais)
*   Hipótese alternativa: \\( H_1: \mu_1 \neq \mu_2 \neq \cdots \neq \mu_k \\) (as médias não são todas iguais)

In [ ]:
# Cria conjunto de dados com pesos (kg) de militares para diferentes patentes
massas = stats.poisson.rvs(loc=10, mu=80, size=1000)
# Conjunto de pesos diferentes para uma das patentes
massas_alt = stats.poisson.rvs(loc=10, mu=75, size=1000)
massas = np.where(respondentes=="TEN_CAP",massas_alt,massas)
# Transforma em dataframe as respostas e respondentes
resultado['peso'] = pd.DataFrame({"peso":massas})

resultado.head()

In [ ]:
# Filtra os pesos dos militares por patente e armazena em novas variáveis
cb_soldados = resultado.peso[resultado['patente']=='CB_SD']
graduados = resultado.peso[resultado['patente']=='SGT_ST']
cap_tenentes = resultado.peso[resultado['patente']=='TEN_CAP']
of_superiores = resultado.peso[resultado['patente']=='OF_SUP']

Usando o módulo `stats`, basta chamar o comando `scipy.stats.f_oneway(*samples, axis=0)`, em que:
*   `*samples` são os dados das medidas para cada categoria (grupo)
*   `axis=0` define se a leitura dos dados se dará por linha ou por coluna

A saída do comando é a estatística do teste (\\(\mathrm{F} )\\) e o valor-p.

In [ ]:
f, p = stats.f_oneway(cb_soldados,graduados,cap_tenentes,of_superiores)
f,p

In [ ]:
# nível de significância
alpha = 0.05
# Imprime o resultado do teste
print(f'A estatística do teste foi F={f:.4f}, correspondente ao valor-p de {p:.3%}.')
if p <= alpha:
  print(f'Hipótese nula rejeitada ao nível de significância de {alpha:.1%}. Portanto, as médias não são todas iguais.')
else:
  print(f'Não há evidência suficiente para rejeitar a hipótese nula em favor da hipótese alternativa. Ou seja, assume-se que as médias são todas iguais.')

## Análise alternativa do resultado do teste (pelo valor da estatística do teste).

<img src="https://analystprep.com/cfa-level-1-exam/wp-content/uploads/2019/08/cfa-level-1-F-distribution.png"  width="50%" height="40%">

In [ ]:
# Graus de liberdade do numerador
k = len(pd.unique(resultado['patente']))
gl_num = k - 1
# Graus de liberdade do denominador
N = len(resultado)
gl_den = N - k
# Valor crítico tabelado de F ao nível de significância definido
ftab = stats.f.ppf(1-alpha, gl_num, gl_den)
ftab

In [ ]:
# Se o valor do teste cair acima do valor crítico tabelado, está na região crítica
if f > ftab:
  print(f'Hipótese nula rejeitada, pois F={f:.4f} está na região crítica ao nível de significância de {alpha:.1%}.')
else:
  print(f'Não há evidência suficiente para rejeitar a hipótese nula, pois F={f:.4f} está na região de aceitação ao nível de significância de {alpha:.1%}.')

## Pós-Teste Tukey

A ANOVA informa se há pelo menos uma das médias diferente entre os grupos sob análise. No entanto, não especifica qual(is) grupo(s) diferem. Então, é costumeiro fazer um *post hoc* teste quando a ANOVA rejeita a hipótese nula para identificar o(s) grupo(s) diferente(s). No caso, vamos utilizar o teste `Tukey`.

In [ ]:
# Importação da biblioteca para o teste Tukey
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
tukey = pairwise_tukeyhsd(endog=resultado['peso'],       # Dados
                          groups=resultado['patente'],   # Grupos
                          alpha=0.05)                    # Nível de significância

tukey.summary()

**Conclusão**: o grupo de capitães e tenentes (TEN_CAP) resultou diferente (True) em relação a todos os demais grupos. 

#Bônus

**Pingouin** é um pacote estatístico de código aberto escrito em Python 3 relativamente novo. Ele foi desenvolvido para usuários que desejam funções de estatísticas simples, mas exaustivas.

A documentação do pacote pode ser consultada em https://pingouin-stats.org/build/html/index.html

![](https://pingouin-stats.org/build/html/_images/logo_pingouin.png)

In [ ]:
# Instalação do pacote
!pip install pingouin

In [ ]:
# Importação da biblioteca
import pingouin as pg

## Teste \\( \chi^2 \\) para independência de variáveis

In [ ]:
expected, observed, stats = pg.chi2_independence(data=resultado, x='patente', y='concordancia')
stats

In [ ]:
observed

In [ ]:
expected

## Teste ANOVA

In [ ]:
pg.anova(data=resultado, dv='peso', between='patente', detailed=True)

## Pós-Teste Tukey

In [ ]:
pg.pairwise_tukey(data=resultado, dv='peso', between='patente')